# Steps

merge
- merge CHR with fips code using 'cbsacode' column
- merge SMART with CHR
- select columns
- aggregate the data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
intermediate_dir = "..\\intermediate\\"
data_dir = "..\\data\\"
today = "20250402"

In [3]:
chr = pd.read_csv(f"{intermediate_dir}{today}combinedCHR_selectedColumns_onlyCalculation.csv")[1:]

In [97]:
smart_cbsa = pd.read_csv(f"{intermediate_dir}smart_with_cbsa_all_12_years.csv")#, usecols=lambda column: column != 'Unnamed: 0') ## change
smart_cbsa['cbsacode'] = smart_cbsa['cbsacode'].astype(int)

In [75]:
smart_cbsa.shape

(876, 8)

In [76]:
chr.head(3)

,county,countycode,fipscode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
1,Autauga County,1,1001,AL,1,2011.0,50756.0,14110.168,1,1,1001.0,AL,Autauga County,2011.0,10040.957968,11543.437080,5167.062312,4177.624848,NaN,7660.0,2143.0,4119.0,1.0,51622.0,19529.0,307.8
2,Baldwin County,3,1003,AL,1,2011.0,179878.0,41551.818,1,3,1003.0,AL,Baldwin County,2011.0,30293.433858,36241.459684,26835.279308,13555.965836,NaN,26292.0,7094.0,11557.0,29.0,51957.0,63562.0,226.4
3,Barbour County,5,1005,AL,1,2011.0,29737.0,6631.351,1,5,1005.0,AL,Barbour County,2011.0,5291.193621,8687.724024,1963.980165,3142.368264,NaN,2869.0,1271.0,3569.0,2.0,30896.0,8347.0,474.3


In [98]:
smart_cbsa.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880
1,10740,2012,"Albuquerque, NM",%,21.22,19.48,22.96,2876
2,10740,2013,"Albuquerque, NM",%,20.25,18.05,22.45,1748


In [80]:
len(smart_cbsa.cbsacode.unique())

73

In [81]:
smart_cbsa.groupby("Year").size()

Year
2011    73
2012    73
2013    73
2014    73
2015    73
2016    73
2017    73
2018    73
2019    73
2020    73
2021    73
2022    73
dtype: int64

In [82]:
smart_cbsa.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size'],
      dtype='object')

## Merge

In [83]:
print(smart_cbsa.shape, chr.shape)

(876, 8) (38320, 26)


In [84]:
chr['fipscode'].nunique()

3200

In [85]:
cbsa = pd.read_csv(f"{data_dir}cbsa2fipsxw.csv")
cbsa['metropolitandivisioncode'] = cbsa['metropolitandivisioncode'].fillna(cbsa['cbsacode']) # brutal force, alternatively. could dropna()
cbsa['fipscode'] = cbsa.fipsstatecode.astype(str) + cbsa.fipscountycode.astype(str).str.zfill(3)

cbsa = cbsa[["fipscode", "cbsacode"]] #, "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis", "fipscode"]]
cbsa.head(3)

,fipscode,cbsacode
0,1001,33860
1,1003,19300
2,1005,21640


In [86]:
chr

,county,countycode,fipscode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
1,Autauga County,1,1001,AL,1,2011.0,50756.0,14110.168000,1,1,1001.0,AL,Autauga County,2011.0,10040.957968,11543.437080,5167.062312,4177.624848,NaN,7660.0,2143.0,4119.0,1.0,51622.0,19529.0,307.8
2,Baldwin County,3,1003,AL,1,2011.0,179878.0,41551.818000,1,3,1003.0,AL,Baldwin County,2011.0,30293.433858,36241.459684,26835.279308,13555.965836,NaN,26292.0,7094.0,11557.0,29.0,51957.0,63562.0,226.4
3,Barbour County,5,1005,AL,1,2011.0,29737.0,6631.351000,1,5,1005.0,AL,Barbour County,2011.0,5291.193621,8687.724024,1963.980165,3142.368264,NaN,2869.0,1271.0,3569.0,2.0,30896.0,8347.0,474.3
4,Bibb County,7,1007,AL,1,2011.0,21587.0,5029.771000,1,7,1007.0,AL,Bibb County,2011.0,5463.885570,5347.984967,1870.966877,1837.852419,NaN,2497.0,1030.0,1635.0,0.0,41076.0,7274.0,300.5
5,Blount County,9,1009,AL,1,2011.0,58345.0,14119.490000,1,9,1009.0,AL,Blount County,2011.0,9862.288730,14107.937690,2167.049990,5041.708140,NaN,6599.0,2348.0,3343.0,1.0,46086.0,19290.0,104.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38317,Sweetwater County,37,56037,WY,56,2022.0,42673.0,10843.000002,56,37,56037.0,WY,Sweetwater County,2022.0,5411.100000,10663.049999,5612.432067,2641.890000,NaN,7490.0,1532.0,1774.0,105.0,70583.0,16120.0,349.5
38318,Teton County,39,56039,WY,56,2022.0,23497.0,4161.000001,56,39,56039.0,WY,Teton County,2022.0,2184.968000,4292.592000,3745.177664,1276.176000,NaN,6344.0,912.0,773.0,128.0,92488.0,9685.0,332.4
38319,Uinta County,41,56041,WY,56,2022.0,20215.0,5608.000000,56,41,56041.0,WY,Uinta County,2022.0,2746.116000,4352.886000,2656.623943,1212.381000,NaN,3015.0,582.0,628.0,74.0,71246.0,7653.0,267.0
38320,Washakie County,43,56043,WY,56,2022.0,7760.0,1706.000000,56,43,56043.0,WY,Washakie County,2022.0,1065.504000,1749.606000,1154.502489,538.806000,NaN,1137.0,211.0,292.0,31.0,58532.0,3182.0,192.2


In [ ]:
print("chr.shape", chr.shape)
chr = chr.dropna(subset=['fipscode']) ## brutal
print("chr.shape", chr.shape)

print(cbsa.shape, chr.shape)

chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

chr_cbsa = pd.merge(
    cbsa, chr, 
    on='fipscode', 
    how='inner' ##
)

chr_cbsa.shape

chr.shape (38320, 26)
chr.shape (38320, 26)
(1915, 2) (38320, 26)


(22020, 27)

In [88]:
# sanity check
chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

merged_df = pd.merge(
    cbsa, chr,
    left_on='fipscode', 
    right_on='fipscode',
    indicator=True 
)

print("merged_df.shape:", merged_df.shape)
merged_df
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']

print("Left-only rows in 'chr':")
print(len(left_only_rows))

merged_df.shape: (22020, 28)
Left-only rows in 'chr':
0


In [89]:
print(chr_cbsa.shape, smart_cbsa.shape)

(22020, 27) (876, 8)


In [90]:
smart_cbsa.cbsacode

0      10740.0
1      10740.0
2      10740.0
3      10740.0
4      10740.0
        ...   
871    49340.0
872    49340.0
873    49340.0
874    49340.0
875    49340.0
Name: cbsacode, Length: 876, dtype: float64

In [91]:
print("Columns in smart_cbsa:", smart_cbsa.columns)
print("Columns in chr:", chr_cbsa.columns)

Columns in smart_cbsa: Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size'],
      dtype='object')
Columns in chr: Index(['fipscode', 'cbsacode', 'county', 'countycode', 'state', 'statecode',
       'year', 'v051_rawvalue', 'v052_rawvalue', 'statecode.1', 'countycode.1',
       'fipscode.1', 'state.1', 'county.1', 'year.1', 'v009_rawvalue',
       'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue', 'v070_numerator',
       'v069_numerator', 'v023_numerator', 'v082_numerator', 'v062_numerator',
       'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')


In [92]:
for i in (chr_cbsa.columns):
    print(i)

fipscode
cbsacode
county
countycode
state
statecode
year
v051_rawvalue
v052_rawvalue
statecode.1
countycode.1
fipscode.1
state.1
county.1
year.1
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [95]:
chr_cbsa

,fipscode,cbsacode,county,countycode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,1001,33860,Autauga County,1,AL,1,2011.0,50756.0,14110.168000,1,1,1001.0,AL,Autauga County,2011.0,10040.957968,11543.437080,5167.062312,4177.624848,NaN,7660.0,2143.0,4119.0,1.0,51622.0,19529.0,307.8
1,1001,33860,Autauga County,1,AL,1,2012.0,50756.0,14110.168000,1,1,1001.0,AL,Autauga County,2012.0,9014.874672,12496.228712,5826.687288,4507.437336,NaN,7941.0,1933.0,4338.0,1.0,53049.0,21279.0,424.9
2,1001,33860,Autauga County,1,AL,1,2013.0,55267.0,14388.000003,1,1,1001.0,AL,Autauga County,2013.0,9606.564999,13939.738999,6745.035000,5028.117000,NaN,7990.0,2076.0,3872.0,2.0,48863.0,21262.0,362.8
3,1001,33860,Autauga County,1,AL,1,2014.0,55514.0,14409.000000,1,1,1001.0,AL,Autauga County,2014.0,8919.785000,12537.025000,6864.535000,4850.390000,11310.0,7940.0,1644.0,4388.0,NaN,51441.0,21393.0,446.9
4,1001,33860,Autauga County,1,AL,1,2015.0,55246.0,14028.999999,1,1,1001.0,AL,Autauga County,2015.0,8944.089000,12900.921000,6883.239000,4987.257000,10974.0,8145.0,1449.0,4229.0,7.0,51868.0,21372.0,508.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22015,56041,21740,Uinta County,41,WY,56,2018.0,20773.0,6098.000001,56,41,56041.0,WY,Uinta County,2018.0,2668.001012,4725.350000,2356.593546,1467.500000,3508.0,3102.0,546.0,1352.0,71.0,64713.0,7049.0,310.9
22016,56041,21740,Uinta County,41,WY,56,2019.0,20495.0,5987.000001,56,41,56041.0,WY,Uinta County,2019.0,2637.639433,4613.544000,2329.775753,1349.244000,3331.0,2955.0,411.0,1310.0,70.0,67404.0,7262.0,288.2
22017,56041,21740,Uinta County,41,WY,56,2020.0,20299.0,5844.000011,56,41,56041.0,WY,Uinta County,2020.0,2488.092112,5174.889996,2762.835781,1633.414999,3841.0,2964.0,390.0,1043.0,72.0,63401.0,7247.0,151.3
22018,56041,21740,Uinta County,41,WY,56,2021.0,20226.0,5752.000000,56,41,56041.0,WY,Uinta County,2021.0,3005.455190,5094.848000,2664.273669,1490.822000,3519.0,2821.0,353.0,602.0,72.0,70756.0,6946.0,204.9


In [99]:
data = pd.merge(
    smart_cbsa, chr_cbsa,
    left_on=['cbsacode', 'Year'],
    right_on=['cbsacode', 'year'],
    how='left'
)
data.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,fipscode,county,countycode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,Bernalillo County,1.0,NM,35.0,2011.0,642527.0,156134.061,35.0,1.0,35001.0,NM,Bernalillo County,2011.0,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,Sandoval County,43.0,NM,35.0,2011.0,125988.0,32756.880,35.0,43.0,35043.0,NM,Sandoval County,2011.0,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9
2,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35057.0,Torrance County,57.0,NM,35.0,2011.0,16475.0,3921.050,35.0,57.0,35057.0,NM,Torrance County,2011.0,4042.371900,2887.408500,1719.891150,765.790950,NaN,1525.0,626.0,1498.0,0.0,36419.0,5102.0,215.1


In [100]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'statecode.1', 'countycode.1',
       'fipscode.1', 'state.1', 'county.1', 'year.1', 'v009_rawvalue',
       'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue', 'v070_numerator',
       'v069_numerator', 'v023_numerator', 'v082_numerator', 'v062_numerator',
       'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [101]:
data = data.loc[:, ~data.columns.str.endswith('.1')]


In [102]:
data.shape

(6060, 28)

In [103]:
data.groupby(['Year']).size()

Year
2011    505
2012    505
2013    505
2014    505
2015    505
2016    505
2017    505
2018    505
2019    505
2020    505
2021    505
2022    505
dtype: int64

In [104]:
data.groupby(['Year', 'cbsacode']).size()

Year  cbsacode
2011  10740        4
      11260        2
      12060       29
      12260        7
      12420        5
                  ..
2022  45820        5
      46140        7
      47260       18
      48620        4
      49340        1
Length: 876, dtype: int64

In [105]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'v009_rawvalue', 'v011_rawvalue',
       'v049_rawvalue', 'v060_rawvalue', 'v070_numerator', 'v069_numerator',
       'v023_numerator', 'v082_numerator', 'v062_numerator', 'v063_rawvalue',
       'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [107]:
for i in data.columns:
    print(i)

cbsacode
Year
cbsatitle
Data_value_unit
Data_value
Confidence_limit_Low
Confidence_limit_High
Sample_Size
fipscode
county
countycode
state
statecode
year
v051_rawvalue
v052_rawvalue
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [108]:
data.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,fipscode,county,countycode,state,statecode,year,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,Bernalillo County,1.0,NM,35.0,2011.0,642527.0,156134.061,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,Sandoval County,43.0,NM,35.0,2011.0,125988.0,32756.880,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9
2,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35057.0,Torrance County,57.0,NM,35.0,2011.0,16475.0,3921.050,4042.371900,2887.408500,1719.891150,765.790950,NaN,1525.0,626.0,1498.0,0.0,36419.0,5102.0,215.1


In [109]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'v009_rawvalue', 'v011_rawvalue',
       'v049_rawvalue', 'v060_rawvalue', 'v070_numerator', 'v069_numerator',
       'v023_numerator', 'v082_numerator', 'v062_numerator', 'v063_rawvalue',
       'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [110]:
columns_to_drop = ['state',
                    'county',
                    'year','countycode']
filtered_df = data.drop(columns=columns_to_drop)

new_column_names = {
    'fipscode': 'smart_fipscode',
}

# Rename columns in filtered_df
filtered_df = filtered_df.rename(columns=new_column_names)


In [111]:
for i in filtered_df.columns:
    print(i)

cbsacode
Year
cbsatitle
Data_value_unit
Data_value
Confidence_limit_Low
Confidence_limit_High
Sample_Size
smart_fipscode
statecode
v051_rawvalue
v052_rawvalue
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [112]:
sum_cols = [col for col in filtered_df.columns if col.startswith('v')]
#sum_cols.remove("v052_rawvalue")  # Remove the rate% of children to pop
sum_cols

['v051_rawvalue',
 'v052_rawvalue',
 'v009_rawvalue',
 'v011_rawvalue',
 'v049_rawvalue',
 'v060_rawvalue',
 'v070_numerator',
 'v069_numerator',
 'v023_numerator',
 'v082_numerator',
 'v062_numerator',
 'v063_rawvalue',
 'v067_numerator',
 'v045_rawvalue']

In [113]:
filtered_df.head(2)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,smart_fipscode,statecode,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,35.0,642527.0,156134.061,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,35.0,125988.0,32756.880,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9


In [114]:
filtered_df.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'smart_fipscode', 'statecode', 'v051_rawvalue', 'v052_rawvalue',
       'v009_rawvalue', 'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue',
       'v070_numerator', 'v069_numerator', 'v023_numerator', 'v082_numerator',
       'v062_numerator', 'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [117]:
def weighted_sum_proportional(group):
    result_value = {}
    for col in sum_cols:
        total_population = group['v051_rawvalue'].fillna(0).sum()
        weighted_sum = group[col].fillna(0).sum()
        sum_population_non_na = group.loc[group[col].notna(), 'v051_rawvalue'].fillna(0).sum()
        result_value[col] = (weighted_sum / sum_population_non_na) * total_population if sum_population_non_na != 0 else 0
    return pd.Series(result_value)


aggregated_df = filtered_df.groupby([
    'Year', "cbsacode", 'Sample_Size', 'Data_value', 'Data_value_unit'
]).apply(weighted_sum_proportional).reset_index()


aggregated_df['Year'] = aggregated_df['Year'].astype(int)
print(aggregated_df.shape)

(876, 19)


In [118]:
aggregated_df

,Year,cbsacode,Sample_Size,Data_value,Data_value_unit,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,2011,10740,2880,21.78,%,857903.0,2.117694e+05,123687.787094,1.357007e+05,85547.993919,38302.509293,0.0,143682.0,30309.0,73613.0,174.0,182856.0,303398.0,1239.8
1,2011,11260,482,17.54,%,374553.0,9.842329e+04,58399.185228,7.407142e+04,49694.181724,18560.182330,0.0,70584.0,14187.0,27308.0,147.0,145971.0,132468.0,1110.0
2,2011,12060,15892,13.45,%,5503952.0,1.483403e+06,688901.435410,1.063977e+06,605445.249041,382898.985792,0.0,1066375.0,260002.0,457387.0,1928.0,1608146.0,1942529.0,8768.2
3,2011,12260,1442,15.61,%,547067.0,1.387515e+05,88740.928716,1.282184e+05,56390.564574,46079.670066,0.0,78056.0,24444.0,56538.0,156.0,298566.0,185331.0,3372.0
4,2011,12420,1205,15.76,%,1705075.0,4.299228e+05,227276.255641,3.056785e+05,258215.965344,89557.563452,0.0,374121.0,61456.0,112860.0,911.0,280085.0,607740.0,2275.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,2022,45820,1345,22.48,%,230878.0,5.369700e+04,30858.951001,6.751237e+04,32694.785366,18218.266001,0.0,36382.0,6889.0,11784.0,660.0,297947.0,89334.0,1987.4
872,2022,46140,2331,28.97,%,1006411.0,2.443900e+05,142723.974003,2.745027e+05,108317.909984,81929.391002,0.0,164229.0,31057.0,63747.0,3933.0,395327.0,377679.0,3138.0
873,2022,47260,8550,22.73,%,1760740.0,3.844730e+05,235602.113995,4.751754e+05,257462.399611,148211.222997,0.0,351364.0,61026.0,119400.0,3528.0,1261482.0,699550.0,11657.6
874,2022,48620,1516,20.64,%,643768.0,1.618890e+05,82246.350996,1.816031e+05,81941.549291,51666.979998,0.0,113437.0,26369.0,39928.0,1477.0,252086.0,253448.0,1747.2


In [119]:
aggregated_df.isna().sum()

Year               0
cbsacode           0
Sample_Size        0
Data_value         0
Data_value_unit    0
v051_rawvalue      0
v052_rawvalue      0
v009_rawvalue      0
v011_rawvalue      0
v049_rawvalue      0
v060_rawvalue      0
v070_numerator     0
v069_numerator     0
v023_numerator     0
v082_numerator     0
v062_numerator     0
v063_rawvalue      0
v067_numerator     0
v045_rawvalue      0
dtype: int64

In [120]:
aggregated_df[aggregated_df.duplicated()]

,Year,cbsacode,Sample_Size,Data_value,Data_value_unit,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue


In [121]:
aggregated_df.cbsacode.value_counts()

10740    12
30780    12
38900    12
38860    12
38300    12
         ..
20260    12
19820    12
19780    12
19740    12
49340    12
Name: cbsacode, Length: 73, dtype: int64

In [122]:
aggregated_df.sort_values(by=['cbsacode'], ascending=True).to_csv(f"{intermediate_dir}{today}agg_smart_chr.csv", index=False)# 

In [ ]:
# aggregated_mean_df = aggregated_df.drop(columns=['Year','Sample_Size', 'Data_value', 'Data_value_unit']).groupby(
#     ['cbsacode']
# ).mean().reset_index()


# (aggregated_mean_df.head())

C:\Users\hongy\AppData\Local\Temp\ipykernel_25084\2970341038.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).mean().reset_index()


,cbsacode,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,9.007533e+05,2.086053e+05,1.153103e+05,1.593776e+05,1.018831e+05,52913.647683,7.539300e+04,1.542459e+05,27696.916667,76940.416667,2426.083333,1.903890e+05,3.193261e+05,1895.494727
1,11260,3.933499e+05,1.002907e+05,5.159039e+04,8.381503e+04,5.812845e+04,21501.950690,3.532275e+04,7.673842e+04,12613.083333,27379.833333,880.333333,1.510622e+05,1.436252e+05,1265.834771
2,12060,1.142628e+07,2.890414e+06,1.332957e+06,2.409281e+06,1.383376e+06,855828.511186,1.207818e+06,2.169286e+06,386403.000000,934087.166667,11903.500000,3.329319e+06,4.029463e+06,20985.253059
3,12260,5.846941e+05,1.390700e+05,8.555323e+04,1.463179e+05,7.027121e+04,54450.385535,7.527833e+04,8.854275e+04,18075.083333,56849.750000,658.333333,3.246662e+05,2.024714e+05,3778.142626
4,12420,1.976510e+06,4.733130e+05,2.068379e+05,3.821950e+05,3.195188e+05,116038.318989,1.626410e+05,4.362552e+05,49525.666667,122148.583333,2810.916667,3.124442e+05,7.229990e+05,2775.115469


In [ ]:
# aggregated_mean_df.shape

(88, 8)

In [ ]:
# duplicates = aggregated_mean_df[aggregated_mean_df.duplicated()]

# if not duplicates.empty:
#     print("Duplicate rows found:")
#     print(duplicates)
# else:
#     print("No duplicates found.")


No duplicates found.


## Save into csv

In [ ]:
## change
# aggregated_mean_df.sort_values(by=['cbsacode'], ascending=True).to_csv(f"{intermediate_dir}{today}agg_smart_chr.csv", index=False)# 